In [ ]:
import tensorflow as tf
import numpy as np
from keras.applications.efficientnet import preprocess_input, EfficientNetB5, EfficientNetB4
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications import MobileNetV3Large, MobileNetV3Small
from keras.applications.efficientnet_v2 import EfficientNetV2B0, EfficientNetV2B1, EfficientNetV2B2, EfficientNetV2B3, EfficientNetV2S, EfficientNetV2M, EfficientNetV2L

# Tạo model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.models import Model

# Tạo và xử lý data
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

# Hàm loss và optimizer
from keras.losses import CategoricalCrossentropy
from keras.optimizers import Adam

# Callbacks
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt

from tabulate import tabulate
# ROC Curve
from sklearn.preprocessing import label_binarize
#Checkpoint and log
from keras.callbacks import CSVLogger
import tempfile
import os
import time
import csv

#Time inference
import cv2


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

# EfficientNetV2B0

In [ ]:
# Xây dựng mô hình
num_classes = 4
IMG_SIZE = 224
batch_size = 64
epochs = 50

inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(weights=None, include_top=True, classes=num_classes) 
predictions = base_model(inputs)

model = Model(inputs=inputs, outputs=predictions)

# Đóng băng các lớp của mô hình base
for layer in base_model.layers:
    layer.trainable = True

model.summary()

In [ ]:
# Đường dẫn đến thư mục chứa dữ liệu
data_dir = 'Data_Cucur_augmented/train'
test_dir = 'Data_Cucur_augmented/test'
# Thiết lập các thông số
validation_ratio = 0.1

# Sử dụng ImageDataGenerator để tạo dữ liệu và áp dụng các biến đổi
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=validation_ratio,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest'
)

# Tạo generator cho tập train và tập validation
train_data_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Chỉ sử dụng tập huấn luyện
)

val_data_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Chỉ sử dụng tập validation
)
test_datagen = ImageDataGenerator(rescale=1 / 255.0)
test_data_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False,
    seed=42
)

In [ ]:
print(len(train_data_generator.filepaths))
print(len(val_data_generator.filepaths))
print(len(test_data_generator.filepaths))

In [ ]:
#save log
class TimeTrainCSVLogger(tf.keras.callbacks.CSVLogger):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
        super().on_epoch_begin(epoch, logs)
    
    def on_epoch_end(self, epoch, logs=None):
        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - self.epoch_start_time
        logs['time_per_epoch'] = epoch_duration
        super().on_epoch_end(epoch, logs)
        print(f"Epoch {epoch + 1} - Training Time: {epoch_duration:.2f} seconds")
        
class TrainingTimeCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        super().on_train_begin(logs)

    def on_train_end(self, logs={}):
        end_time = time.time()
        duration = end_time - self.start_time
        super().on_train_end(logs)
        hours = duration // 3600
        minutes = (duration - (hours * 3600)) // 60
        seconds = duration - ((hours * 3600) + (minutes * 60))
        
        msg = f'Total Training Time: {int(hours)} hours, {int(minutes)} minutes, {seconds:.2f} seconds'
        print(msg)

        # Ghi tổng thời gian huấn luyện vào tệp CSV
        with open('efficientNetV2B0/log/train_logger1.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Total Training Time (seconds)'])
            writer.writerow([duration])
            writer.writerow(['Total Training Time (hours, minutes, seconds)'])
            writer.writerow([hours, minutes, seconds])

csv_logger = TimeTrainCSVLogger('efficientNetV2B0/log/train_logger1.csv', append=True, separator='\t')

checkpoint_filepath = "efficientNetV2B0/checkpoint/cp1.ckpt"
directory_checkpoint = os.path.dirname(checkpoint_filepath)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose=2,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=False,
    save_freq='epoch')


In [ ]:
# Optimizer và loss function
loss = CategoricalCrossentropy()
optimizer = Adam(learning_rate=0.0001)
model.compile(loss=loss, optimizer=optimizer, metrics= ['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

# Tính toán số bước cho tập train và tập validation
steps_per_epoch = train_data_generator.samples // batch_size
validation_steps = val_data_generator.samples // batch_size

history = model.fit(train_data_generator,
          steps_per_epoch=steps_per_epoch,
          validation_steps= validation_steps, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=val_data_generator, 
          callbacks=[model_checkpoint_callback,csv_logger,TrainingTimeCallback()]
          )

In [ ]:
# Đánh giá mô hình trên tệp test
test_loss, test_accuracy, test_recall, test_precision = model.evaluate(test_data_generator)

print(f'Test_loss: {test_loss}')
print(f'Test_accuracy: {test_accuracy}')
print(f'Test_recall: {test_recall}')
print(f'Test_precision: {test_precision}')

with open('efficientNetV2B0/Test_data_metrics1.txt', 'w') as file:
    file.write(f'Test_loss: {test_loss}\n')
    file.write(f'Test_accuracy: {test_accuracy}\n')
    file.write(f'Test_recall: {test_recall}\n')
    file.write(f'Test_precision: {test_precision}\n')

# Lưu mô hình
model.save('efficientNetV2B0/efficientnet_v2_b0_model.h5')

In [ ]:
# Lấy thông tin về độ chính xác trên tập huấn luyện và tập validation
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Vẽ đồ thị biểu diễn sự thay đổi của accuracy
plt.plot(range(1, epochs + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, epochs + 1), val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')
plt.legend()
plt.savefig('efficientNetV2B0/accuracy_vs_epochs_v2b0.png') # save before show. If not, it will save a blank image
plt.show()

# Lấy thông tin về loss trên tập huấn luyện và tập validation
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Vẽ đồ thị biểu diễn sự thay đổi của loss
plt.plot(range(1, epochs + 1), train_loss, label='Training Loss')
plt.plot(range(1, epochs + 1), val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')
plt.legend()
plt.savefig('efficientNetV2B0/loss_vs_epochs_v2b0.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'precision' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Precision vs Epochs')
plt.legend()
plt.savefig('efficientNetV2B0/precision_vs_epochs_v2b0.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('efficientNetV2B0/recall_vs_epochs_v2b0.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('efficientNetV2B0/recall_vs_epochs_v2b0.png') # save before show. If not, it will save a blank image
plt.show()

# Dự đoán trên tập kiểm tra và lấy nhãn dự đoán
y_pred = model.predict(test_data_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Lấy nhãn thật của tập kiểm tra
y_true_indices = test_data_generator.classes

# Lấy danh sách các lớp từ tên thư mục trong val_data_generator
class_names = list(test_data_generator.class_indices.keys())

# Sử dụng chỉ số thứ tự để lấy tên của các lớp từ danh sách tên lớp
y_true_classes = [class_names[i] for i in y_true_indices]

# Tính toán confusion matrix
cm = confusion_matrix(y_true_indices, y_pred_labels)

# Hiển thị confusion matrix bằng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('efficientNetV2B0/confusion_matrix_v2b0.png') # save before show. If not, it will save a blank image
plt.show()

# Tính classification report
class_names = list(test_data_generator.class_indices.keys())
report = classification_report(y_true_indices, y_pred_labels, target_names=class_names, output_dict=True)

# Tạo bảng classification report
report_table = []
for class_name, metrics in report.items():
    if class_name in class_names:
        row = [class_name, metrics['precision'], metrics['recall'], metrics['f1-score']]
        report_table.append(row)

headers = ["Class", "Precision", "Recall", "F1-Score"]
report_text = tabulate(report_table, headers, tablefmt="pretty")
# Lưu bảng classification report vào tệp văn bản
with open('efficientNetV2B0/classification_report_v2b0.txt', 'w') as file:
    file.write(report_text)

# In bảng classification report ra màn hình
print(report_text)

# Chuyển đổi nhãn thành định dạng mã hóa one-hot
y_test_binary = label_binarize(test_data_generator.labels, classes=range(len(class_names)))

# Khởi tạo một từ điển trống để lưu FPR và TPR cho mỗi lớp
fpr = {}
tpr = {}
roc_auc = {}

# Tạo một hình ảnh cho các đường ROC
plt.figure(figsize=(8, 6))

# Duyệt qua từng lớp
for i in range(len(class_names)):
    # Tính toán đường ROC và AUC cho lớp hiện tại
    fpr[i], tpr[i], _ = roc_curve(y_test_binary[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

    # Vẽ đường ROC cho lớp hiện tại
    plt.plot(fpr[i], tpr[i], lw=2, label=f'ROC curve for {class_names[i]} (area = {roc_auc[i]:.2f})')

# Cài đặt plot
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.savefig('efficientNetV2B0/ROC_curve_v2b0.png') # save before show. If not, it will save a blank image
plt.show()

# MobileNetV2


In [ ]:
# Xây dựng mô hình
num_classes = 4
IMG_SIZE = 224
batch_size = 64
epochs = 50

inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(weights=None, include_top=True, classes=num_classes) 
predictions = base_model(inputs)

model = Model(inputs=inputs, outputs=predictions)

# Đóng băng các lớp của mô hình base
for layer in base_model.layers:
    layer.trainable = True

model.summary()

In [ ]:
#save log
class TimeTrainCSVLogger(tf.keras.callbacks.CSVLogger):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
        super().on_epoch_begin(epoch, logs)
    
    def on_epoch_end(self, epoch, logs=None):
        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - self.epoch_start_time
        logs['time_per_epoch'] = epoch_duration
        super().on_epoch_end(epoch, logs)
        print(f"Epoch {epoch + 1} - Training Time: {epoch_duration:.2f} seconds")
        
class TrainingTimeCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        super().on_train_begin(logs)

    def on_train_end(self, logs={}):
        end_time = time.time()
        duration = end_time - self.start_time
        super().on_train_end(logs)
        hours = duration // 3600
        minutes = (duration - (hours * 3600)) // 60
        seconds = duration - ((hours * 3600) + (minutes * 60))
        
        msg = f'Total Training Time: {int(hours)} hours, {int(minutes)} minutes, {seconds:.2f} seconds'
        print(msg)

        # Ghi tổng thời gian huấn luyện vào tệp CSV
        with open('mobileNetV2/log/train_logger2.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Total Training Time (seconds)'])
            writer.writerow([duration])
            writer.writerow(['Total Training Time (hours, minutes, seconds)'])
            writer.writerow([hours, minutes, seconds])

csv_logger = TimeTrainCSVLogger('mobileNetV2/log/train_logger2.csv', append=True, separator='\t')

checkpoint_filepath = "mobileNetV2/checkpoint/cp2.ckpt"
directory_checkpoint = os.path.dirname(checkpoint_filepath)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose=2,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=False,
    save_freq='epoch')


In [ ]:
# Optimizer và loss function
loss = CategoricalCrossentropy()
optimizer = Adam(learning_rate=0.0001)
model.compile(loss=loss, optimizer=optimizer, metrics= ['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

# Tính toán số bước cho tập train và tập validation
steps_per_epoch = train_data_generator.samples // batch_size
validation_steps = val_data_generator.samples // batch_size

history = model.fit(train_data_generator,
          steps_per_epoch=steps_per_epoch,
          validation_steps= validation_steps, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=val_data_generator, 
          callbacks=[model_checkpoint_callback,csv_logger,TrainingTimeCallback()]
          )

In [ ]:
# Đánh giá mô hình trên tệp test
test_loss, test_accuracy, test_recall, test_precision = model.evaluate(test_data_generator)

print(f'Test_loss: {test_loss}')
print(f'Test_accuracy: {test_accuracy}')
print(f'Test_recall: {test_recall}')
print(f'Test_precision: {test_precision}')

with open('mobileNetV2/Test_data_metrics2.txt', 'w') as file:
    file.write(f'Test_loss: {test_loss}\n')
    file.write(f'Test_accuracy: {test_accuracy}\n')
    file.write(f'Test_recall: {test_recall}\n')
    file.write(f'Test_precision: {test_precision}\n')

# Lưu mô hình
model.save('mobileNetV2/mobileNetV2_model.h5')

In [ ]:
# Lấy thông tin về độ chính xác trên tập huấn luyện và tập validation
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Vẽ đồ thị biểu diễn sự thay đổi của accuracy
plt.plot(range(1, epochs + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, epochs + 1), val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')
plt.legend()
plt.savefig('mobileNetV2/accuracy_vs_epochs_mobileNetV2.png') # save before show. If not, it will save a blank image
plt.show()

# Lấy thông tin về loss trên tập huấn luyện và tập validation
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Vẽ đồ thị biểu diễn sự thay đổi của loss
plt.plot(range(1, epochs + 1), train_loss, label='Training Loss')
plt.plot(range(1, epochs + 1), val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')
plt.legend()
plt.savefig('mobileNetV2/loss_vs_epochs_mobileNetV2.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'precision' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Precision vs Epochs')
plt.legend()
plt.savefig('mobileNetV2/precision_vs_epochs_mobileNetV2.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('mobileNetV2/recall_vs_epochs_mobileNetV2.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('mobileNetV2/recall_vs_epochs_mobileNetV2.png') # save before show. If not, it will save a blank image
plt.show()

# Dự đoán trên tập kiểm tra và lấy nhãn dự đoán
y_pred = model.predict(test_data_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Lấy nhãn thật của tập kiểm tra
y_true_indices = test_data_generator.classes

# Lấy danh sách các lớp từ tên thư mục trong val_data_generator
class_names = list(test_data_generator.class_indices.keys())

# Sử dụng chỉ số thứ tự để lấy tên của các lớp từ danh sách tên lớp
y_true_classes = [class_names[i] for i in y_true_indices]

# Tính toán confusion matrix
cm = confusion_matrix(y_true_indices, y_pred_labels)

# Hiển thị confusion matrix bằng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('mobileNetV2/confusion_matrix_mobileNetV2.png') # save before show. If not, it will save a blank image
plt.show()

# Tính classification report
class_names = list(test_data_generator.class_indices.keys())
report = classification_report(y_true_indices, y_pred_labels, target_names=class_names, output_dict=True)

# Tạo bảng classification report
report_table = []
for class_name, metrics in report.items():
    if class_name in class_names:
        row = [class_name, metrics['precision'], metrics['recall'], metrics['f1-score']]
        report_table.append(row)

headers = ["Class", "Precision", "Recall", "F1-Score"]
report_text = tabulate(report_table, headers, tablefmt="pretty")
# Lưu bảng classification report vào tệp văn bản
with open('mobileNetV2/classification_report_mobileNetV2.txt', 'w') as file:
    file.write(report_text)

# In bảng classification report ra màn hình
print(report_text)

# Chuyển đổi nhãn thành định dạng mã hóa one-hot
y_test_binary = label_binarize(test_data_generator.labels, classes=range(len(class_names)))

# Khởi tạo một từ điển trống để lưu FPR và TPR cho mỗi lớp
fpr = {}
tpr = {}
roc_auc = {}

# Tạo một hình ảnh cho các đường ROC
plt.figure(figsize=(8, 6))

# Duyệt qua từng lớp
for i in range(len(class_names)):
    # Tính toán đường ROC và AUC cho lớp hiện tại
    fpr[i], tpr[i], _ = roc_curve(y_test_binary[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

    # Vẽ đường ROC cho lớp hiện tại
    plt.plot(fpr[i], tpr[i], lw=2, label=f'ROC curve for {class_names[i]} (area = {roc_auc[i]:.2f})')

# Cài đặt plot
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.savefig('mobileNetV2/ROC_curve_mobileNetV2.png') # save before show. If not, it will save a blank image
plt.show()

# MobileNetV3Large


In [ ]:
# Xây dựng mô hình
num_classes = 4
IMG_SIZE = 224
batch_size = 64
epochs = 50

inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = tf.keras.applications.MobileNetV3Large(weights=None, include_top=True, classes=num_classes, dropout_rate=0.2) 
predictions = base_model(inputs)

model = Model(inputs=inputs, outputs=predictions)

# Đóng băng các lớp của mô hình base
for layer in base_model.layers:
    layer.trainable = True

model.summary()

In [ ]:
#save log
class TimeTrainCSVLogger(tf.keras.callbacks.CSVLogger):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
        super().on_epoch_begin(epoch, logs)
    
    def on_epoch_end(self, epoch, logs=None):
        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - self.epoch_start_time
        logs['time_per_epoch'] = epoch_duration
        super().on_epoch_end(epoch, logs)
        print(f"Epoch {epoch + 1} - Training Time: {epoch_duration:.2f} seconds")
        
class TrainingTimeCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        super().on_train_begin(logs)

    def on_train_end(self, logs={}):
        end_time = time.time()
        duration = end_time - self.start_time
        super().on_train_end(logs)
        hours = duration // 3600
        minutes = (duration - (hours * 3600)) // 60
        seconds = duration - ((hours * 3600) + (minutes * 60))
        
        msg = f'Total Training Time: {int(hours)} hours, {int(minutes)} minutes, {seconds:.2f} seconds'
        print(msg)

        # Ghi tổng thời gian huấn luyện vào tệp CSV
        with open('mobileNetV3Large/log/train_logger3.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Total Training Time (seconds)'])
            writer.writerow([duration])
            writer.writerow(['Total Training Time (hours, minutes, seconds)'])
            writer.writerow([hours, minutes, seconds])

csv_logger = TimeTrainCSVLogger('mobileNetV3Large/log/train_logger3.csv', append=True, separator='\t')

checkpoint_filepath = "mobileNetV3Large/checkpoint/cp3.ckpt"
directory_checkpoint = os.path.dirname(checkpoint_filepath)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose=2,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=False,
    save_freq='epoch')


In [ ]:
# Optimizer và loss function
loss = CategoricalCrossentropy()
optimizer = Adam(learning_rate=0.0001)
model.compile(loss=loss, optimizer=optimizer, metrics= ['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

# Tính toán số bước cho tập train và tập validation
steps_per_epoch = train_data_generator.samples // batch_size
validation_steps = val_data_generator.samples // batch_size

history = model.fit(train_data_generator,
          steps_per_epoch=steps_per_epoch,
          validation_steps= validation_steps, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=val_data_generator, 
          callbacks=[model_checkpoint_callback,csv_logger,TrainingTimeCallback()]
          )

In [ ]:
# Đánh giá mô hình trên tệp test
test_loss, test_accuracy, test_recall, test_precision = model.evaluate(test_data_generator)

print(f'Test_loss: {test_loss}')
print(f'Test_accuracy: {test_accuracy}')
print(f'Test_recall: {test_recall}')
print(f'Test_precision: {test_precision}')

with open('mobileNetV3Large/Test_data_metrics3.txt', 'w') as file:
    file.write(f'Test_loss: {test_loss}\n')
    file.write(f'Test_accuracy: {test_accuracy}\n')
    file.write(f'Test_recall: {test_recall}\n')
    file.write(f'Test_precision: {test_precision}\n')

# Lưu mô hình
model.save('mobileNetV3Large/mobileNetV3Large_model.h5')

In [ ]:
# Lấy thông tin về độ chính xác trên tập huấn luyện và tập validation
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Vẽ đồ thị biểu diễn sự thay đổi của accuracy
plt.plot(range(1, epochs + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, epochs + 1), val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')
plt.legend()
plt.savefig('mobileNetV3Large/accuracy_vs_epochs_mv3l.png') # save before show. If not, it will save a blank image
plt.show()

# Lấy thông tin về loss trên tập huấn luyện và tập validation
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Vẽ đồ thị biểu diễn sự thay đổi của loss
plt.plot(range(1, epochs + 1), train_loss, label='Training Loss')
plt.plot(range(1, epochs + 1), val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')
plt.legend()
plt.savefig('mobileNetV3Large/loss_vs_epochs_mv3l.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'precision' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Precision vs Epochs')
plt.legend()
plt.savefig('mobileNetV3Large/precision_vs_epochs_mv3l.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('mobileNetV3Large/recall_vs_epochs_mv3l.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('mobileNetV3Large/recall_vs_epochs_mv3l.png') # save before show. If not, it will save a blank image
plt.show()

# Dự đoán trên tập kiểm tra và lấy nhãn dự đoán
y_pred = model.predict(test_data_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Lấy nhãn thật của tập kiểm tra
y_true_indices = test_data_generator.classes

# Lấy danh sách các lớp từ tên thư mục trong val_data_generator
class_names = list(test_data_generator.class_indices.keys())

# Sử dụng chỉ số thứ tự để lấy tên của các lớp từ danh sách tên lớp
y_true_classes = [class_names[i] for i in y_true_indices]

# Tính toán confusion matrix
cm = confusion_matrix(y_true_indices, y_pred_labels)

# Hiển thị confusion matrix bằng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('mobileNetV3Large/confusion_matrix_mv3l.png') # save before show. If not, it will save a blank image
plt.show()

# Tính classification report
class_names = list(test_data_generator.class_indices.keys())
report = classification_report(y_true_indices, y_pred_labels, target_names=class_names, output_dict=True)

# Tạo bảng classification report
report_table = []
for class_name, metrics in report.items():
    if class_name in class_names:
        row = [class_name, metrics['precision'], metrics['recall'], metrics['f1-score']]
        report_table.append(row)

headers = ["Class", "Precision", "Recall", "F1-Score"]
report_text = tabulate(report_table, headers, tablefmt="pretty")
# Lưu bảng classification report vào tệp văn bản
with open('mobileNetV3Large/classification_report_mv3l.txt', 'w') as file:
    file.write(report_text)

# In bảng classification report ra màn hình
print(report_text)

# Chuyển đổi nhãn thành định dạng mã hóa one-hot
y_test_binary = label_binarize(test_data_generator.labels, classes=range(len(class_names)))

# Khởi tạo một từ điển trống để lưu FPR và TPR cho mỗi lớp
fpr = {}
tpr = {}
roc_auc = {}

# Tạo một hình ảnh cho các đường ROC
plt.figure(figsize=(8, 6))

# Duyệt qua từng lớp
for i in range(len(class_names)):
    # Tính toán đường ROC và AUC cho lớp hiện tại
    fpr[i], tpr[i], _ = roc_curve(y_test_binary[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

    # Vẽ đường ROC cho lớp hiện tại
    plt.plot(fpr[i], tpr[i], lw=2, label=f'ROC curve for {class_names[i]} (area = {roc_auc[i]:.2f})')

# Cài đặt plot
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.savefig('mobileNetV3Large/ROC_curve_mv3l.png') # save before show. If not, it will save a blank image
plt.show()

# MobileNetV3Small

In [ ]:
# Xây dựng mô hình
num_classes = 4
IMG_SIZE = 224
batch_size = 64
epochs = 50

inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = tf.keras.applications.MobileNetV3Small(weights=None, include_top=True, classes=num_classes, dropout_rate=0.2)
predictions = base_model(inputs)

model = Model(inputs=inputs, outputs=predictions)

# Đóng băng các lớp của mô hình base
for layer in base_model.layers:
    layer.trainable = True

model.summary()

In [ ]:
#save log
class TimeTrainCSVLogger(tf.keras.callbacks.CSVLogger):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
        super().on_epoch_begin(epoch, logs)
    
    def on_epoch_end(self, epoch, logs=None):
        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - self.epoch_start_time
        logs['time_per_epoch'] = epoch_duration
        super().on_epoch_end(epoch, logs)
        print(f"Epoch {epoch + 1} - Training Time: {epoch_duration:.2f} seconds")
        
class TrainingTimeCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        super().on_train_begin(logs)

    def on_train_end(self, logs={}):
        end_time = time.time()
        duration = end_time - self.start_time
        super().on_train_end(logs)
        hours = duration // 3600
        minutes = (duration - (hours * 3600)) // 60
        seconds = duration - ((hours * 3600) + (minutes * 60))
        
        msg = f'Total Training Time: {int(hours)} hours, {int(minutes)} minutes, {seconds:.2f} seconds'
        print(msg)

        # Ghi tổng thời gian huấn luyện vào tệp CSV
        with open('mobileNetV3Small/log/train_logger4.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Total Training Time (seconds)'])
            writer.writerow([duration])
            writer.writerow(['Total Training Time (hours, minutes, seconds)'])
            writer.writerow([hours, minutes, seconds])

csv_logger = TimeTrainCSVLogger('mobileNetV3Small/log/train_logger4.csv', append=True, separator='\t')

checkpoint_filepath = "mobileNetV3Small/checkpoint/cp4.ckpt"
directory_checkpoint = os.path.dirname(checkpoint_filepath)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose=2,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=False,
    save_freq='epoch')


In [ ]:
# Optimizer và loss function
loss = CategoricalCrossentropy()
optimizer = Adam(learning_rate=0.0001)
model.compile(loss=loss, optimizer=optimizer, metrics= ['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

# Tính toán số bước cho tập train và tập validation
steps_per_epoch = train_data_generator.samples // batch_size
validation_steps = val_data_generator.samples // batch_size

history = model.fit(train_data_generator,
          steps_per_epoch=steps_per_epoch,
          validation_steps= validation_steps, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=val_data_generator, 
          callbacks=[model_checkpoint_callback,csv_logger,TrainingTimeCallback()]
          )

In [ ]:
# Đánh giá mô hình trên tệp test
test_loss, test_accuracy, test_recall, test_precision = model.evaluate(test_data_generator)

print(f'Test_loss: {test_loss}')
print(f'Test_accuracy: {test_accuracy}')
print(f'Test_recall: {test_recall}')
print(f'Test_precision: {test_precision}')

with open('mobileNetV3Small/Test_data_metrics4.txt', 'w') as file:
    file.write(f'Test_loss: {test_loss}\n')
    file.write(f'Test_accuracy: {test_accuracy}\n')
    file.write(f'Test_recall: {test_recall}\n')
    file.write(f'Test_precision: {test_precision}\n')

# Lưu mô hình
model.save('mobileNetV3Small/mobileNetV3Small_model.h5')

In [ ]:
# Lấy thông tin về độ chính xác trên tập huấn luyện và tập validation
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Vẽ đồ thị biểu diễn sự thay đổi của accuracy
plt.plot(range(1, epochs + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, epochs + 1), val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')
plt.legend()
plt.savefig('mobileNetV3Small/accuracy_vs_epochs_4.png') # save before show. If not, it will save a blank image
plt.show()

# Lấy thông tin về loss trên tập huấn luyện và tập validation
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Vẽ đồ thị biểu diễn sự thay đổi của loss
plt.plot(range(1, epochs + 1), train_loss, label='Training Loss')
plt.plot(range(1, epochs + 1), val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')
plt.legend()
plt.savefig('mobileNetV3Small/loss_vs_epochs_4.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'precision' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Precision vs Epochs')
plt.legend()
plt.savefig('mobileNetV3Small/precision_vs_epochs_4.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('mobileNetV3Small/recall_vs_epochs_4.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('mobileNetV3Small/recall_vs_epochs_4.png') # save before show. If not, it will save a blank image
plt.show()

# Dự đoán trên tập kiểm tra và lấy nhãn dự đoán
y_pred = model.predict(test_data_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Lấy nhãn thật của tập kiểm tra
y_true_indices = test_data_generator.classes

# Lấy danh sách các lớp từ tên thư mục trong val_data_generator
class_names = list(test_data_generator.class_indices.keys())

# Sử dụng chỉ số thứ tự để lấy tên của các lớp từ danh sách tên lớp
y_true_classes = [class_names[i] for i in y_true_indices]

# Tính toán confusion matrix
cm = confusion_matrix(y_true_indices, y_pred_labels)

# Hiển thị confusion matrix bằng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('mobileNetV3Small/confusion_matrix_4.png') # save before show. If not, it will save a blank image
plt.show()

# Tính classification report
class_names = list(test_data_generator.class_indices.keys())
report = classification_report(y_true_indices, y_pred_labels, target_names=class_names, output_dict=True)

# Tạo bảng classification report
report_table = []
for class_name, metrics in report.items():
    if class_name in class_names:
        row = [class_name, metrics['precision'], metrics['recall'], metrics['f1-score']]
        report_table.append(row)

headers = ["Class", "Precision", "Recall", "F1-Score"]
report_text = tabulate(report_table, headers, tablefmt="pretty")
# Lưu bảng classification report vào tệp văn bản
with open('mobileNetV3Small/classification_report_4.txt', 'w') as file:
    file.write(report_text)

# In bảng classification report ra màn hình
print(report_text)

# Chuyển đổi nhãn thành định dạng mã hóa one-hot
y_test_binary = label_binarize(test_data_generator.labels, classes=range(len(class_names)))

# Khởi tạo một từ điển trống để lưu FPR và TPR cho mỗi lớp
fpr = {}
tpr = {}
roc_auc = {}

# Tạo một hình ảnh cho các đường ROC
plt.figure(figsize=(8, 6))

# Duyệt qua từng lớp
for i in range(len(class_names)):
    # Tính toán đường ROC và AUC cho lớp hiện tại
    fpr[i], tpr[i], _ = roc_curve(y_test_binary[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

    # Vẽ đường ROC cho lớp hiện tại
    plt.plot(fpr[i], tpr[i], lw=2, label=f'ROC curve for {class_names[i]} (area = {roc_auc[i]:.2f})')

# Cài đặt plot
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.savefig('mobileNetV3Small/ROC_curve_4.png') # save before show. If not, it will save a blank image
plt.show()

# EfficientNetV2M

In [ ]:
# Xây dựng mô hình
num_classes = 4
IMG_SIZE = 480
batch_size = 64
epochs = 50

inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2M(weights=None, include_top=True, classes=num_classes) 
predictions = base_model(inputs)

model = Model(inputs=inputs, outputs=predictions)

# Đóng băng các lớp của mô hình base
for layer in base_model.layers:
    layer.trainable = True

model.summary()

In [ ]:
# Đường dẫn đến thư mục chứa dữ liệu
data_dir = 'Data_Cucur_augmented/train'
test_dir = 'Data_Cucur_augmented/test'
# Thiết lập các thông số
validation_ratio = 0.1

# Sử dụng ImageDataGenerator để tạo dữ liệu và áp dụng các biến đổi
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=validation_ratio,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest'
)

# Tạo generator cho tập train và tập validation
train_data_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Chỉ sử dụng tập huấn luyện
)

val_data_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Chỉ sử dụng tập validation
)
test_datagen = ImageDataGenerator(rescale=1 / 255.0)
test_data_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False,
    seed=42
)

In [ ]:
print(len(train_data_generator.filepaths))
print(len(val_data_generator.filepaths))
print(len(test_data_generator.filepaths))

In [ ]:
#save log
class TimeTrainCSVLogger(tf.keras.callbacks.CSVLogger):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
        super().on_epoch_begin(epoch, logs)
    
    def on_epoch_end(self, epoch, logs=None):
        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - self.epoch_start_time
        logs['time_per_epoch'] = epoch_duration
        super().on_epoch_end(epoch, logs)
        print(f"Epoch {epoch + 1} - Training Time: {epoch_duration:.2f} seconds")
        
class TrainingTimeCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        super().on_train_begin(logs)

    def on_train_end(self, logs={}):
        end_time = time.time()
        duration = end_time - self.start_time
        super().on_train_end(logs)
        hours = duration // 3600
        minutes = (duration - (hours * 3600)) // 60
        seconds = duration - ((hours * 3600) + (minutes * 60))
        
        msg = f'Total Training Time: {int(hours)} hours, {int(minutes)} minutes, {seconds:.2f} seconds'
        print(msg)

        # Ghi tổng thời gian huấn luyện vào tệp CSV
        with open('efficientNetV2M/log/train_logger5.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Total Training Time (seconds)'])
            writer.writerow([duration])
            writer.writerow(['Total Training Time (hours, minutes, seconds)'])
            writer.writerow([hours, minutes, seconds])

csv_logger = TimeTrainCSVLogger('efficientNetV2M/log/train_logger5.csv', append=True, separator='\t')

checkpoint_filepath = "efficientNetV2M/checkpoint/cp5.ckpt"
directory_checkpoint = os.path.dirname(checkpoint_filepath)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose=2,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=False,
    save_freq='epoch')


In [ ]:
# Optimizer và loss function
loss = CategoricalCrossentropy()
optimizer = Adam(learning_rate=0.0001)
model.compile(loss=loss, optimizer=optimizer, metrics= ['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

# Tính toán số bước cho tập train và tập validation
steps_per_epoch = train_data_generator.samples // batch_size
validation_steps = val_data_generator.samples // batch_size

history = model.fit(train_data_generator,
          steps_per_epoch=steps_per_epoch,
          validation_steps= validation_steps, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=val_data_generator, 
          callbacks=[model_checkpoint_callback,csv_logger,TrainingTimeCallback()]
          )

In [ ]:
# Đánh giá mô hình trên tệp test
test_loss, test_accuracy, test_recall, test_precision = model.evaluate(test_data_generator)

print(f'Test_loss: {test_loss}')
print(f'Test_accuracy: {test_accuracy}')
print(f'Test_recall: {test_recall}')
print(f'Test_precision: {test_precision}')

with open('efficientNetV2M/Test_data_metrics5.txt', 'w') as file:
    file.write(f'Test_loss: {test_loss}\n')
    file.write(f'Test_accuracy: {test_accuracy}\n')
    file.write(f'Test_recall: {test_recall}\n')
    file.write(f'Test_precision: {test_precision}\n')

# Lưu mô hình
model.save('efficientNetV2M/efficientNetV2M_model.h5')

In [ ]:
# Lấy thông tin về độ chính xác trên tập huấn luyện và tập validation
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Vẽ đồ thị biểu diễn sự thay đổi của accuracy
plt.plot(range(1, epochs + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, epochs + 1), val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')
plt.legend()
plt.savefig('efficientNetV2M/accuracy_vs_epochs_5.png') # save before show. If not, it will save a blank image
plt.show()

# Lấy thông tin về loss trên tập huấn luyện và tập validation
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Vẽ đồ thị biểu diễn sự thay đổi của loss
plt.plot(range(1, epochs + 1), train_loss, label='Training Loss')
plt.plot(range(1, epochs + 1), val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')
plt.legend()
plt.savefig('efficientNetV2M/loss_vs_epochs_5.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'precision' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Precision vs Epochs')
plt.legend()
plt.savefig('efficientNetV2M/precision_vs_epochs_5.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('efficientNetV2M/recall_vs_epochs_5.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('efficientNetV2M/recall_vs_epochs_5.png') # save before show. If not, it will save a blank image
plt.show()

# Dự đoán trên tập kiểm tra và lấy nhãn dự đoán
y_pred = model.predict(test_data_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Lấy nhãn thật của tập kiểm tra
y_true_indices = test_data_generator.classes

# Lấy danh sách các lớp từ tên thư mục trong val_data_generator
class_names = list(test_data_generator.class_indices.keys())

# Sử dụng chỉ số thứ tự để lấy tên của các lớp từ danh sách tên lớp
y_true_classes = [class_names[i] for i in y_true_indices]

# Tính toán confusion matrix
cm = confusion_matrix(y_true_indices, y_pred_labels)

# Hiển thị confusion matrix bằng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('efficientNetV2M/confusion_matrix_5.png') # save before show. If not, it will save a blank image
plt.show()

# Tính classification report
class_names = list(test_data_generator.class_indices.keys())
report = classification_report(y_true_indices, y_pred_labels, target_names=class_names, output_dict=True)

# Tạo bảng classification report
report_table = []
for class_name, metrics in report.items():
    if class_name in class_names:
        row = [class_name, metrics['precision'], metrics['recall'], metrics['f1-score']]
        report_table.append(row)

headers = ["Class", "Precision", "Recall", "F1-Score"]
report_text = tabulate(report_table, headers, tablefmt="pretty")
# Lưu bảng classification report vào tệp văn bản
with open('efficientNetV2M/classification_report_5.txt', 'w') as file:
    file.write(report_text)

# In bảng classification report ra màn hình
print(report_text)

# Chuyển đổi nhãn thành định dạng mã hóa one-hot
y_test_binary = label_binarize(test_data_generator.labels, classes=range(len(class_names)))

# Khởi tạo một từ điển trống để lưu FPR và TPR cho mỗi lớp
fpr = {}
tpr = {}
roc_auc = {}

# Tạo một hình ảnh cho các đường ROC
plt.figure(figsize=(8, 6))

# Duyệt qua từng lớp
for i in range(len(class_names)):
    # Tính toán đường ROC và AUC cho lớp hiện tại
    fpr[i], tpr[i], _ = roc_curve(y_test_binary[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

    # Vẽ đường ROC cho lớp hiện tại
    plt.plot(fpr[i], tpr[i], lw=2, label=f'ROC curve for {class_names[i]} (area = {roc_auc[i]:.2f})')

# Cài đặt plot
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.savefig('efficientNetV2M/ROC_curve_5.png') # save before show. If not, it will save a blank image
plt.show()

# EfficientNetV2L

In [ ]:
# Xây dựng mô hình
num_classes = 4
IMG_SIZE = 480
batch_size = 64
epochs = 50

inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2L(weights=None, include_top=True, classes=num_classes) 
predictions = base_model(inputs)

model = Model(inputs=inputs, outputs=predictions)

# Đóng băng các lớp của mô hình base
for layer in base_model.layers:
    layer.trainable = True

model.summary()

In [ ]:
#save log
class TimeTrainCSVLogger(tf.keras.callbacks.CSVLogger):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
        super().on_epoch_begin(epoch, logs)
    
    def on_epoch_end(self, epoch, logs=None):
        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - self.epoch_start_time
        logs['time_per_epoch'] = epoch_duration
        super().on_epoch_end(epoch, logs)
        print(f"Epoch {epoch + 1} - Training Time: {epoch_duration:.2f} seconds")
        
class TrainingTimeCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        super().on_train_begin(logs)

    def on_train_end(self, logs={}):
        end_time = time.time()
        duration = end_time - self.start_time
        super().on_train_end(logs)
        hours = duration // 3600
        minutes = (duration - (hours * 3600)) // 60
        seconds = duration - ((hours * 3600) + (minutes * 60))
        
        msg = f'Total Training Time: {int(hours)} hours, {int(minutes)} minutes, {seconds:.2f} seconds'
        print(msg)

        # Ghi tổng thời gian huấn luyện vào tệp CSV
        with open('efficientNetV2L/log/train_logger6.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Total Training Time (seconds)'])
            writer.writerow([duration])
            writer.writerow(['Total Training Time (hours, minutes, seconds)'])
            writer.writerow([hours, minutes, seconds])

csv_logger = TimeTrainCSVLogger('efficientNetV2L/log/train_logger6.csv', append=True, separator='\t')

checkpoint_filepath = "efficientNetV2L/checkpoint/cp6.ckpt"
directory_checkpoint = os.path.dirname(checkpoint_filepath)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose=2,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=False,
    save_freq='epoch')


In [ ]:
# Optimizer và loss function
loss = CategoricalCrossentropy()
optimizer = Adam(learning_rate=0.0001)
model.compile(loss=loss, optimizer=optimizer, metrics= ['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

# Tính toán số bước cho tập train và tập validation
steps_per_epoch = train_data_generator.samples // batch_size
validation_steps = val_data_generator.samples // batch_size

history = model.fit(train_data_generator,
          steps_per_epoch=steps_per_epoch,
          validation_steps= validation_steps, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=val_data_generator, 
          callbacks=[model_checkpoint_callback,csv_logger,TrainingTimeCallback()]
          )

In [ ]:
# Đánh giá mô hình trên tệp test
test_loss, test_accuracy, test_recall, test_precision = model.evaluate(test_data_generator)

print(f'Test_loss: {test_loss}')
print(f'Test_accuracy: {test_accuracy}')
print(f'Test_recall: {test_recall}')
print(f'Test_precision: {test_precision}')

with open('efficientNetV2L/Test_data_metrics.txt', 'w') as file:
    file.write(f'Test_loss: {test_loss}\n')
    file.write(f'Test_accuracy: {test_accuracy}\n')
    file.write(f'Test_recall: {test_recall}\n')
    file.write(f'Test_precision: {test_precision}\n')

# Lưu mô hình
model.save('efficientNetV2L/efficientNetV2L_model.h5')

In [ ]:
# Lấy thông tin về độ chính xác trên tập huấn luyện và tập validation
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Vẽ đồ thị biểu diễn sự thay đổi của accuracy
plt.plot(range(1, epochs + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, epochs + 1), val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')
plt.legend()
plt.savefig('efficientNetV2L/accuracy_vs_epochs_6.png') # save before show. If not, it will save a blank image
plt.show()

# Lấy thông tin về loss trên tập huấn luyện và tập validation
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Vẽ đồ thị biểu diễn sự thay đổi của loss
plt.plot(range(1, epochs + 1), train_loss, label='Training Loss')
plt.plot(range(1, epochs + 1), val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')
plt.legend()
plt.savefig('efficientNetV2L/loss_vs_epochs_6.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'precision' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Precision vs Epochs')
plt.legend()
plt.savefig('efficientNetV2L/precision_vs_epochs_6.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('efficientNetV2L/recall_vs_epochs_6.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('efficientNetV2L/recall_vs_epochs_6.png') # save before show. If not, it will save a blank image
plt.show()

# Dự đoán trên tập kiểm tra và lấy nhãn dự đoán
y_pred = model.predict(test_data_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Lấy nhãn thật của tập kiểm tra
y_true_indices = test_data_generator.classes

# Lấy danh sách các lớp từ tên thư mục trong val_data_generator
class_names = list(test_data_generator.class_indices.keys())

# Sử dụng chỉ số thứ tự để lấy tên của các lớp từ danh sách tên lớp
y_true_classes = [class_names[i] for i in y_true_indices]

# Tính toán confusion matrix
cm = confusion_matrix(y_true_indices, y_pred_labels)

# Hiển thị confusion matrix bằng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('efficientNetV2L/confusion_matrix_6.png') # save before show. If not, it will save a blank image
plt.show()

# Tính classification report
class_names = list(test_data_generator.class_indices.keys())
report = classification_report(y_true_indices, y_pred_labels, target_names=class_names, output_dict=True)

# Tạo bảng classification report
report_table = []
for class_name, metrics in report.items():
    if class_name in class_names:
        row = [class_name, metrics['precision'], metrics['recall'], metrics['f1-score']]
        report_table.append(row)

headers = ["Class", "Precision", "Recall", "F1-Score"]
report_text = tabulate(report_table, headers, tablefmt="pretty")
# Lưu bảng classification report vào tệp văn bản
with open('efficientNetV2L/classification_report_6.txt', 'w') as file:
    file.write(report_text)

# In bảng classification report ra màn hình
print(report_text)

# Chuyển đổi nhãn thành định dạng mã hóa one-hot
y_test_binary = label_binarize(test_data_generator.labels, classes=range(len(class_names)))

# Khởi tạo một từ điển trống để lưu FPR và TPR cho mỗi lớp
fpr = {}
tpr = {}
roc_auc = {}

# Tạo một hình ảnh cho các đường ROC
plt.figure(figsize=(8, 6))

# Duyệt qua từng lớp
for i in range(len(class_names)):
    # Tính toán đường ROC và AUC cho lớp hiện tại
    fpr[i], tpr[i], _ = roc_curve(y_test_binary[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

    # Vẽ đường ROC cho lớp hiện tại
    plt.plot(fpr[i], tpr[i], lw=2, label=f'ROC curve for {class_names[i]} (area = {roc_auc[i]:.2f})')

# Cài đặt plot
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.savefig('efficientNetV2L/ROC_curve_6.png') # save before show. If not, it will save a blank image
plt.show()

# EfficientNetV2B1

In [ ]:
# Xây dựng mô hình
num_classes = 4
IMG_SIZE = 240
batch_size = 64
epochs = 50

inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B1(weights=None, include_top=True, classes=num_classes) 
predictions = base_model(inputs)

model = Model(inputs=inputs, outputs=predictions)

# Đóng băng các lớp của mô hình base
for layer in base_model.layers:
    layer.trainable = True

model.summary()

In [ ]:
# Đường dẫn đến thư mục chứa dữ liệu
data_dir = 'Data_Cucur_augmented/train'
test_dir = 'Data_Cucur_augmented/test'
# Thiết lập các thông số
validation_ratio = 0.1

# Sử dụng ImageDataGenerator để tạo dữ liệu và áp dụng các biến đổi
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=validation_ratio,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest'
)

# Tạo generator cho tập train và tập validation
train_data_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Chỉ sử dụng tập huấn luyện
)

val_data_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Chỉ sử dụng tập validation
)
test_datagen = ImageDataGenerator(rescale=1 / 255.0)
test_data_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False,
    seed=42
)

In [ ]:
print(len(train_data_generator.filepaths))
print(len(val_data_generator.filepaths))
print(len(test_data_generator.filepaths))

In [ ]:
#save log
class TimeTrainCSVLogger(tf.keras.callbacks.CSVLogger):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
        super().on_epoch_begin(epoch, logs)
    
    def on_epoch_end(self, epoch, logs=None):
        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - self.epoch_start_time
        logs['time_per_epoch'] = epoch_duration
        super().on_epoch_end(epoch, logs)
        print(f"Epoch {epoch + 1} - Training Time: {epoch_duration:.2f} seconds")
        
class TrainingTimeCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        super().on_train_begin(logs)

    def on_train_end(self, logs={}):
        end_time = time.time()
        duration = end_time - self.start_time
        super().on_train_end(logs)
        hours = duration // 3600
        minutes = (duration - (hours * 3600)) // 60
        seconds = duration - ((hours * 3600) + (minutes * 60))
        
        msg = f'Total Training Time: {int(hours)} hours, {int(minutes)} minutes, {seconds:.2f} seconds'
        print(msg)

        # Ghi tổng thời gian huấn luyện vào tệp CSV
        with open('efficientNetV2B1/log/train_logger7.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Total Training Time (seconds)'])
            writer.writerow([duration])
            writer.writerow(['Total Training Time (hours, minutes, seconds)'])
            writer.writerow([hours, minutes, seconds])

csv_logger = TimeTrainCSVLogger('efficientNetV2B1/log/train_logger7.csv', append=True, separator='\t')

checkpoint_filepath = "efficientNetV2B1/checkpoint/cp7.ckpt"
directory_checkpoint = os.path.dirname(checkpoint_filepath)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose=2,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=False,
    save_freq='epoch')


In [ ]:
# Optimizer và loss function
loss = CategoricalCrossentropy()
optimizer = Adam(learning_rate=0.0001)
model.compile(loss=loss, optimizer=optimizer, metrics= ['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

# Tính toán số bước cho tập train và tập validation
steps_per_epoch = train_data_generator.samples // batch_size
validation_steps = val_data_generator.samples // batch_size

history = model.fit(train_data_generator,
          steps_per_epoch=steps_per_epoch,
          validation_steps= validation_steps, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=val_data_generator, 
          callbacks=[model_checkpoint_callback,csv_logger,TrainingTimeCallback()]
          )

In [ ]:
# Đánh giá mô hình trên tệp test
test_loss, test_accuracy, test_recall, test_precision = model.evaluate(test_data_generator)

print(f'Test_loss: {test_loss}')
print(f'Test_accuracy: {test_accuracy}')
print(f'Test_recall: {test_recall}')
print(f'Test_precision: {test_precision}')

with open('efficientNetV2B1/Test_data_metrics.txt', 'w') as file:
    file.write(f'Test_loss: {test_loss}\n')
    file.write(f'Test_accuracy: {test_accuracy}\n')
    file.write(f'Test_recall: {test_recall}\n')
    file.write(f'Test_precision: {test_precision}\n')

# Lưu mô hình
model.save('efficientNetV2B1/efficientNetV2B1_model.h5')

In [ ]:
# Lấy thông tin về độ chính xác trên tập huấn luyện và tập validation
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Vẽ đồ thị biểu diễn sự thay đổi của accuracy
plt.plot(range(1, epochs + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, epochs + 1), val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')
plt.legend()
plt.savefig('efficientNetV2B1/accuracy_vs_epochs_7.png') # save before show. If not, it will save a blank image
plt.show()

# Lấy thông tin về loss trên tập huấn luyện và tập validation
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Vẽ đồ thị biểu diễn sự thay đổi của loss
plt.plot(range(1, epochs + 1), train_loss, label='Training Loss')
plt.plot(range(1, epochs + 1), val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')
plt.legend()
plt.savefig('efficientNetV2B1/loss_vs_epochs_7.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'precision' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Precision vs Epochs')
plt.legend()
plt.savefig('efficientNetV2B1/precision_vs_epochs_7.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('efficientNetV2B1/recall_vs_epochs_7.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('efficientNetV2B1/recall_vs_epochs_7.png') # save before show. If not, it will save a blank image
plt.show()

# Dự đoán trên tập kiểm tra và lấy nhãn dự đoán
y_pred = model.predict(test_data_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Lấy nhãn thật của tập kiểm tra
y_true_indices = test_data_generator.classes

# Lấy danh sách các lớp từ tên thư mục trong val_data_generator
class_names = list(test_data_generator.class_indices.keys())

# Sử dụng chỉ số thứ tự để lấy tên của các lớp từ danh sách tên lớp
y_true_classes = [class_names[i] for i in y_true_indices]

# Tính toán confusion matrix
cm = confusion_matrix(y_true_indices, y_pred_labels)

# Hiển thị confusion matrix bằng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('efficientNetV2B1/confusion_matrix_7.png') # save before show. If not, it will save a blank image
plt.show()

# Tính classification report
class_names = list(test_data_generator.class_indices.keys())
report = classification_report(y_true_indices, y_pred_labels, target_names=class_names, output_dict=True)

# Tạo bảng classification report
report_table = []
for class_name, metrics in report.items():
    if class_name in class_names:
        row = [class_name, metrics['precision'], metrics['recall'], metrics['f1-score']]
        report_table.append(row)

headers = ["Class", "Precision", "Recall", "F1-Score"]
report_text = tabulate(report_table, headers, tablefmt="pretty")
# Lưu bảng classification report vào tệp văn bản
with open('efficientNetV2B1/classification_report_7.txt', 'w') as file:
    file.write(report_text)

# In bảng classification report ra màn hình
print(report_text)

# Chuyển đổi nhãn thành định dạng mã hóa one-hot
y_test_binary = label_binarize(test_data_generator.labels, classes=range(len(class_names)))

# Khởi tạo một từ điển trống để lưu FPR và TPR cho mỗi lớp
fpr = {}
tpr = {}
roc_auc = {}

# Tạo một hình ảnh cho các đường ROC
plt.figure(figsize=(8, 6))

# Duyệt qua từng lớp
for i in range(len(class_names)):
    # Tính toán đường ROC và AUC cho lớp hiện tại
    fpr[i], tpr[i], _ = roc_curve(y_test_binary[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

    # Vẽ đường ROC cho lớp hiện tại
    plt.plot(fpr[i], tpr[i], lw=2, label=f'ROC curve for {class_names[i]} (area = {roc_auc[i]:.2f})')

# Cài đặt plot
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.savefig('efficientNetV2B1/ROC_curve_7.png') # save before show. If not, it will save a blank image
plt.show()

# EfficientNetV2B2

In [ ]:
# Xây dựng mô hình
num_classes = 4
IMG_SIZE = 260
batch_size = 64
epochs = 50

inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B2(weights=None, include_top=True, classes=num_classes) 
predictions = base_model(inputs)

model = Model(inputs=inputs, outputs=predictions)

# Đóng băng các lớp của mô hình base
for layer in base_model.layers:
    layer.trainable = True

model.summary()

In [ ]:
# Đường dẫn đến thư mục chứa dữ liệu
data_dir = 'Data_Cucur_augmented/train'
test_dir = 'Data_Cucur_augmented/test'
# Thiết lập các thông số
validation_ratio = 0.1

# Sử dụng ImageDataGenerator để tạo dữ liệu và áp dụng các biến đổi
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=validation_ratio,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest'
)

# Tạo generator cho tập train và tập validation
train_data_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Chỉ sử dụng tập huấn luyện
)

val_data_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Chỉ sử dụng tập validation
)
test_datagen = ImageDataGenerator(rescale=1 / 255.0)
test_data_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False,
    seed=42
)

In [ ]:
print(len(train_data_generator.filepaths))
print(len(val_data_generator.filepaths))
print(len(test_data_generator.filepaths))

In [ ]:
#save log
class TimeTrainCSVLogger(tf.keras.callbacks.CSVLogger):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
        super().on_epoch_begin(epoch, logs)
    
    def on_epoch_end(self, epoch, logs=None):
        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - self.epoch_start_time
        logs['time_per_epoch'] = epoch_duration
        super().on_epoch_end(epoch, logs)
        print(f"Epoch {epoch + 1} - Training Time: {epoch_duration:.2f} seconds")
        
class TrainingTimeCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        super().on_train_begin(logs)

    def on_train_end(self, logs={}):
        end_time = time.time()
        duration = end_time - self.start_time
        super().on_train_end(logs)
        hours = duration // 3600
        minutes = (duration - (hours * 3600)) // 60
        seconds = duration - ((hours * 3600) + (minutes * 60))
        
        msg = f'Total Training Time: {int(hours)} hours, {int(minutes)} minutes, {seconds:.2f} seconds'
        print(msg)

        # Ghi tổng thời gian huấn luyện vào tệp CSV
        with open('efficientNetV2B2/log/train_logger8.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Total Training Time (seconds)'])
            writer.writerow([duration])
            writer.writerow(['Total Training Time (hours, minutes, seconds)'])
            writer.writerow([hours, minutes, seconds])

csv_logger = TimeTrainCSVLogger('efficientNetV2B2/log/train_logger8.csv', append=True, separator='\t')

checkpoint_filepath = "efficientNetV2B2/checkpoint/cp8.ckpt"
directory_checkpoint = os.path.dirname(checkpoint_filepath)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose=2,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=False,
    save_freq='epoch')


In [ ]:
# Optimizer và loss function
loss = CategoricalCrossentropy()
optimizer = Adam(learning_rate=0.0001)
model.compile(loss=loss, optimizer=optimizer, metrics= ['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

# Tính toán số bước cho tập train và tập validation
steps_per_epoch = train_data_generator.samples // batch_size
validation_steps = val_data_generator.samples // batch_size

history = model.fit(train_data_generator,
          steps_per_epoch=steps_per_epoch,
          validation_steps= validation_steps, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=val_data_generator, 
          callbacks=[model_checkpoint_callback,csv_logger,TrainingTimeCallback()]
          )

In [ ]:
# Đánh giá mô hình trên tệp test
test_loss, test_accuracy, test_recall, test_precision = model.evaluate(test_data_generator)

print(f'Test_loss: {test_loss}')
print(f'Test_accuracy: {test_accuracy}')
print(f'Test_recall: {test_recall}')
print(f'Test_precision: {test_precision}')

with open('efficientNetV2B2/Test_data_metrics.txt', 'w') as file:
    file.write(f'Test_loss: {test_loss}\n')
    file.write(f'Test_accuracy: {test_accuracy}\n')
    file.write(f'Test_recall: {test_recall}\n')
    file.write(f'Test_precision: {test_precision}\n')

# Lưu mô hình
model.save('efficientNetV2B2/efficientNetV2B2_model.h5')

In [ ]:
# Lấy thông tin về độ chính xác trên tập huấn luyện và tập validation
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Vẽ đồ thị biểu diễn sự thay đổi của accuracy
plt.plot(range(1, epochs + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, epochs + 1), val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')
plt.legend()
plt.savefig('efficientNetV2B2/accuracy_vs_epochs_8.png') # save before show. If not, it will save a blank image
plt.show()

# Lấy thông tin về loss trên tập huấn luyện và tập validation
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Vẽ đồ thị biểu diễn sự thay đổi của loss
plt.plot(range(1, epochs + 1), train_loss, label='Training Loss')
plt.plot(range(1, epochs + 1), val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')
plt.legend()
plt.savefig('efficientNetV2B2/loss_vs_epochs_8.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'precision' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Precision vs Epochs')
plt.legend()
plt.savefig('efficientNetV2B2/precision_vs_epochs_8.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('efficientNetV2B2/recall_vs_epochs_8.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('efficientNetV2B2/recall_vs_epochs_8.png') # save before show. If not, it will save a blank image
plt.show()

# Dự đoán trên tập kiểm tra và lấy nhãn dự đoán
y_pred = model.predict(test_data_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Lấy nhãn thật của tập kiểm tra
y_true_indices = test_data_generator.classes

# Lấy danh sách các lớp từ tên thư mục trong val_data_generator
class_names = list(test_data_generator.class_indices.keys())

# Sử dụng chỉ số thứ tự để lấy tên của các lớp từ danh sách tên lớp
y_true_classes = [class_names[i] for i in y_true_indices]

# Tính toán confusion matrix
cm = confusion_matrix(y_true_indices, y_pred_labels)

# Hiển thị confusion matrix bằng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('efficientNetV2B2/confusion_matrix_8.png') # save before show. If not, it will save a blank image
plt.show()

# Tính classification report
class_names = list(test_data_generator.class_indices.keys())
report = classification_report(y_true_indices, y_pred_labels, target_names=class_names, output_dict=True)

# Tạo bảng classification report
report_table = []
for class_name, metrics in report.items():
    if class_name in class_names:
        row = [class_name, metrics['precision'], metrics['recall'], metrics['f1-score']]
        report_table.append(row)

headers = ["Class", "Precision", "Recall", "F1-Score"]
report_text = tabulate(report_table, headers, tablefmt="pretty")
# Lưu bảng classification report vào tệp văn bản
with open('efficientNetV2B2/classification_report_8.txt', 'w') as file:
    file.write(report_text)

# In bảng classification report ra màn hình
print(report_text)

# Chuyển đổi nhãn thành định dạng mã hóa one-hot
y_test_binary = label_binarize(test_data_generator.labels, classes=range(len(class_names)))

# Khởi tạo một từ điển trống để lưu FPR và TPR cho mỗi lớp
fpr = {}
tpr = {}
roc_auc = {}

# Tạo một hình ảnh cho các đường ROC
plt.figure(figsize=(8, 6))

# Duyệt qua từng lớp
for i in range(len(class_names)):
    # Tính toán đường ROC và AUC cho lớp hiện tại
    fpr[i], tpr[i], _ = roc_curve(y_test_binary[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

    # Vẽ đường ROC cho lớp hiện tại
    plt.plot(fpr[i], tpr[i], lw=2, label=f'ROC curve for {class_names[i]} (area = {roc_auc[i]:.2f})')

# Cài đặt plot
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.savefig('efficientNetV2B2/ROC_curve_8.png') # save before show. If not, it will save a blank image
plt.show()

# EfficientNetV2B3

In [ ]:
# Xây dựng mô hình
num_classes = 4
IMG_SIZE = 300
batch_size = 64
epochs = 50

inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B3(weights=None, include_top=True, classes=num_classes) 
predictions = base_model(inputs)

model = Model(inputs=inputs, outputs=predictions)

# Đóng băng các lớp của mô hình base
for layer in base_model.layers:
    layer.trainable = True

model.summary()

In [ ]:
# Đường dẫn đến thư mục chứa dữ liệu
data_dir = 'Data_Cucur_augmented/train'
test_dir = 'Data_Cucur_augmented/test'
# Thiết lập các thông số
validation_ratio = 0.1

# Sử dụng ImageDataGenerator để tạo dữ liệu và áp dụng các biến đổi
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=validation_ratio,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest'
)

# Tạo generator cho tập train và tập validation
train_data_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Chỉ sử dụng tập huấn luyện
)

val_data_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Chỉ sử dụng tập validation
)
test_datagen = ImageDataGenerator(rescale=1 / 255.0)
test_data_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False,
    seed=42
)

In [ ]:
print(len(train_data_generator.filepaths))
print(len(val_data_generator.filepaths))
print(len(test_data_generator.filepaths))

In [ ]:
#save log
class TimeTrainCSVLogger(tf.keras.callbacks.CSVLogger):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
        super().on_epoch_begin(epoch, logs)
    
    def on_epoch_end(self, epoch, logs=None):
        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - self.epoch_start_time
        logs['time_per_epoch'] = epoch_duration
        super().on_epoch_end(epoch, logs)
        print(f"Epoch {epoch + 1} - Training Time: {epoch_duration:.2f} seconds")
        
class TrainingTimeCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        super().on_train_begin(logs)

    def on_train_end(self, logs={}):
        end_time = time.time()
        duration = end_time - self.start_time
        super().on_train_end(logs)
        hours = duration // 3600
        minutes = (duration - (hours * 3600)) // 60
        seconds = duration - ((hours * 3600) + (minutes * 60))
        
        msg = f'Total Training Time: {int(hours)} hours, {int(minutes)} minutes, {seconds:.2f} seconds'
        print(msg)

        # Ghi tổng thời gian huấn luyện vào tệp CSV
        with open('efficientNetV2B3/log/train_logger9.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Total Training Time (seconds)'])
            writer.writerow([duration])
            writer.writerow(['Total Training Time (hours, minutes, seconds)'])
            writer.writerow([hours, minutes, seconds])

csv_logger = TimeTrainCSVLogger('efficientNetV2B3/log/train_logger9.csv', append=True, separator='\t')

checkpoint_filepath = "efficientNetV2B3/checkpoint/cp9.ckpt"
directory_checkpoint = os.path.dirname(checkpoint_filepath)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose=2,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=False,
    save_freq='epoch')


In [ ]:
# Optimizer và loss function
loss = CategoricalCrossentropy()
optimizer = Adam(learning_rate=0.0001)
model.compile(loss=loss, optimizer=optimizer, metrics= ['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

# Tính toán số bước cho tập train và tập validation
steps_per_epoch = train_data_generator.samples // batch_size
validation_steps = val_data_generator.samples // batch_size

history = model.fit(train_data_generator,
          steps_per_epoch=steps_per_epoch,
          validation_steps= validation_steps, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=val_data_generator, 
          callbacks=[model_checkpoint_callback,csv_logger,TrainingTimeCallback()]
          )

In [ ]:
# Đánh giá mô hình trên tệp test
test_loss, test_accuracy, test_recall, test_precision = model.evaluate(test_data_generator)

print(f'Test_loss: {test_loss}')
print(f'Test_accuracy: {test_accuracy}')
print(f'Test_recall: {test_recall}')
print(f'Test_precision: {test_precision}')

with open('efficientNetV2B3/Test_data_metrics.txt', 'w') as file:
    file.write(f'Test_loss: {test_loss}\n')
    file.write(f'Test_accuracy: {test_accuracy}\n')
    file.write(f'Test_recall: {test_recall}\n')
    file.write(f'Test_precision: {test_precision}\n')

# Lưu mô hình
model.save('efficientNetV2B3/efficientNetV2B3_model.h5')

In [ ]:
# Lấy thông tin về độ chính xác trên tập huấn luyện và tập validation
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Vẽ đồ thị biểu diễn sự thay đổi của accuracy
plt.plot(range(1, epochs + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, epochs + 1), val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')
plt.legend()
plt.savefig('efficientNetV2B3/accuracy_vs_epochs_9.png') # save before show. If not, it will save a blank image
plt.show()

# Lấy thông tin về loss trên tập huấn luyện và tập validation
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Vẽ đồ thị biểu diễn sự thay đổi của loss
plt.plot(range(1, epochs + 1), train_loss, label='Training Loss')
plt.plot(range(1, epochs + 1), val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')
plt.legend()
plt.savefig('efficientNetV2B3/loss_vs_epochs_9.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'precision' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Precision vs Epochs')
plt.legend()
plt.savefig('efficientNetV2B3/precision_vs_epochs_9.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('efficientNetV2B3/recall_vs_epochs_9.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('efficientNetV2B3/recall_vs_epochs_9.png') # save before show. If not, it will save a blank image
plt.show()

# Dự đoán trên tập kiểm tra và lấy nhãn dự đoán
y_pred = model.predict(test_data_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Lấy nhãn thật của tập kiểm tra
y_true_indices = test_data_generator.classes

# Lấy danh sách các lớp từ tên thư mục trong val_data_generator
class_names = list(test_data_generator.class_indices.keys())

# Sử dụng chỉ số thứ tự để lấy tên của các lớp từ danh sách tên lớp
y_true_classes = [class_names[i] for i in y_true_indices]

# Tính toán confusion matrix
cm = confusion_matrix(y_true_indices, y_pred_labels)

# Hiển thị confusion matrix bằng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('efficientNetV2B3/confusion_matrix_9.png') # save before show. If not, it will save a blank image
plt.show()

# Tính classification report
class_names = list(test_data_generator.class_indices.keys())
report = classification_report(y_true_indices, y_pred_labels, target_names=class_names, output_dict=True)

# Tạo bảng classification report
report_table = []
for class_name, metrics in report.items():
    if class_name in class_names:
        row = [class_name, metrics['precision'], metrics['recall'], metrics['f1-score']]
        report_table.append(row)

headers = ["Class", "Precision", "Recall", "F1-Score"]
report_text = tabulate(report_table, headers, tablefmt="pretty")
# Lưu bảng classification report vào tệp văn bản
with open('efficientNetV2B3/classification_report_9.txt', 'w') as file:
    file.write(report_text)

# In bảng classification report ra màn hình
print(report_text)

# Chuyển đổi nhãn thành định dạng mã hóa one-hot
y_test_binary = label_binarize(test_data_generator.labels, classes=range(len(class_names)))

# Khởi tạo một từ điển trống để lưu FPR và TPR cho mỗi lớp
fpr = {}
tpr = {}
roc_auc = {}

# Tạo một hình ảnh cho các đường ROC
plt.figure(figsize=(8, 6))

# Duyệt qua từng lớp
for i in range(len(class_names)):
    # Tính toán đường ROC và AUC cho lớp hiện tại
    fpr[i], tpr[i], _ = roc_curve(y_test_binary[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

    # Vẽ đường ROC cho lớp hiện tại
    plt.plot(fpr[i], tpr[i], lw=2, label=f'ROC curve for {class_names[i]} (area = {roc_auc[i]:.2f})')

# Cài đặt plot
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.savefig('efficientNetV2B3/ROC_curve_9.png') # save before show. If not, it will save a blank image
plt.show()

# EfficientNetV2S

In [ ]:
# Xây dựng mô hình
num_classes = 4
IMG_SIZE = 384
batch_size = 64
epochs = 50

inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2S(weights=None, include_top=True, classes=num_classes) 
predictions = base_model(inputs)

model = Model(inputs=inputs, outputs=predictions)

# Đóng băng các lớp của mô hình base
for layer in base_model.layers:
    layer.trainable = True

model.summary()

In [ ]:
# Đường dẫn đến thư mục chứa dữ liệu
data_dir = 'Data_Cucur_augmented/train'
test_dir = 'Data_Cucur_augmented/test'
# Thiết lập các thông số
validation_ratio = 0.1

# Sử dụng ImageDataGenerator để tạo dữ liệu và áp dụng các biến đổi
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=validation_ratio,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest'
)

# Tạo generator cho tập train và tập validation
train_data_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Chỉ sử dụng tập huấn luyện
)

val_data_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Chỉ sử dụng tập validation
)
test_datagen = ImageDataGenerator(rescale=1 / 255.0)
test_data_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False,
    seed=42
)

In [ ]:
print(len(train_data_generator.filepaths))
print(len(val_data_generator.filepaths))
print(len(test_data_generator.filepaths))

In [ ]:
#save log
class TimeTrainCSVLogger(tf.keras.callbacks.CSVLogger):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
        super().on_epoch_begin(epoch, logs)
    
    def on_epoch_end(self, epoch, logs=None):
        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - self.epoch_start_time
        logs['time_per_epoch'] = epoch_duration
        super().on_epoch_end(epoch, logs)
        print(f"Epoch {epoch + 1} - Training Time: {epoch_duration:.2f} seconds")
        
class TrainingTimeCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        super().on_train_begin(logs)

    def on_train_end(self, logs={}):
        end_time = time.time()
        duration = end_time - self.start_time
        super().on_train_end(logs)
        hours = duration // 3600
        minutes = (duration - (hours * 3600)) // 60
        seconds = duration - ((hours * 3600) + (minutes * 60))
        
        msg = f'Total Training Time: {int(hours)} hours, {int(minutes)} minutes, {seconds:.2f} seconds'
        print(msg)

        # Ghi tổng thời gian huấn luyện vào tệp CSV
        with open('efficientNetV2S/log/train_logger10.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Total Training Time (seconds)'])
            writer.writerow([duration])
            writer.writerow(['Total Training Time (hours, minutes, seconds)'])
            writer.writerow([hours, minutes, seconds])

csv_logger = TimeTrainCSVLogger('efficientNetV2S/log/train_logger10.csv', append=True, separator='\t')

checkpoint_filepath = "efficientNetV2S/checkpoint/cp10.ckpt"
directory_checkpoint = os.path.dirname(checkpoint_filepath)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose=2,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=False,
    save_freq='epoch')


In [ ]:
# Optimizer và loss function
loss = CategoricalCrossentropy()
optimizer = Adam(learning_rate=0.0001)
model.compile(loss=loss, optimizer=optimizer, metrics= ['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

# Tính toán số bước cho tập train và tập validation
steps_per_epoch = train_data_generator.samples // batch_size
validation_steps = val_data_generator.samples // batch_size

history = model.fit(train_data_generator,
          steps_per_epoch=steps_per_epoch,
          validation_steps= validation_steps, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=val_data_generator, 
          callbacks=[model_checkpoint_callback,csv_logger,TrainingTimeCallback()]
          )

In [ ]:
# Đánh giá mô hình trên tệp test
test_loss, test_accuracy, test_recall, test_precision = model.evaluate(test_data_generator)

print(f'Test_loss: {test_loss}')
print(f'Test_accuracy: {test_accuracy}')
print(f'Test_recall: {test_recall}')
print(f'Test_precision: {test_precision}')

with open('efficientNetV2S/Test_data_metrics.txt', 'w') as file:
    file.write(f'Test_loss: {test_loss}\n')
    file.write(f'Test_accuracy: {test_accuracy}\n')
    file.write(f'Test_recall: {test_recall}\n')
    file.write(f'Test_precision: {test_precision}\n')

# Lưu mô hình
model.save('efficientNetV2S/efficientNetV2S_model.h5')

In [ ]:
# Lấy thông tin về độ chính xác trên tập huấn luyện và tập validation
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Vẽ đồ thị biểu diễn sự thay đổi của accuracy
plt.plot(range(1, epochs + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, epochs + 1), val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')
plt.legend()
plt.savefig('efficientNetV2S/accuracy_vs_epochs_10.png') # save before show. If not, it will save a blank image
plt.show()

# Lấy thông tin về loss trên tập huấn luyện và tập validation
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Vẽ đồ thị biểu diễn sự thay đổi của loss
plt.plot(range(1, epochs + 1), train_loss, label='Training Loss')
plt.plot(range(1, epochs + 1), val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')
plt.legend()
plt.savefig('efficientNetV2S/loss_vs_epochs_10.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'precision' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Precision vs Epochs')
plt.legend()
plt.savefig('efficientNetV2S/precision_vs_epochs_10.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('efficientNetV2S/recall_vs_epochs_10.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('efficientNetV2S/recall_vs_epochs_10.png') # save before show. If not, it will save a blank image
plt.show()

# Dự đoán trên tập kiểm tra và lấy nhãn dự đoán
y_pred = model.predict(test_data_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Lấy nhãn thật của tập kiểm tra
y_true_indices = test_data_generator.classes

# Lấy danh sách các lớp từ tên thư mục trong val_data_generator
class_names = list(test_data_generator.class_indices.keys())

# Sử dụng chỉ số thứ tự để lấy tên của các lớp từ danh sách tên lớp
y_true_classes = [class_names[i] for i in y_true_indices]

# Tính toán confusion matrix
cm = confusion_matrix(y_true_indices, y_pred_labels)

# Hiển thị confusion matrix bằng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('efficientNetV2S/confusion_matrix_10.png') # save before show. If not, it will save a blank image
plt.show()

# Tính classification report
class_names = list(test_data_generator.class_indices.keys())
report = classification_report(y_true_indices, y_pred_labels, target_names=class_names, output_dict=True)

# Tạo bảng classification report
report_table = []
for class_name, metrics in report.items():
    if class_name in class_names:
        row = [class_name, metrics['precision'], metrics['recall'], metrics['f1-score']]
        report_table.append(row)

headers = ["Class", "Precision", "Recall", "F1-Score"]
report_text = tabulate(report_table, headers, tablefmt="pretty")
# Lưu bảng classification report vào tệp văn bản
with open('efficientNetV2S/classification_report_10.txt', 'w') as file:
    file.write(report_text)

# In bảng classification report ra màn hình
print(report_text)

# Chuyển đổi nhãn thành định dạng mã hóa one-hot
y_test_binary = label_binarize(test_data_generator.labels, classes=range(len(class_names)))

# Khởi tạo một từ điển trống để lưu FPR và TPR cho mỗi lớp
fpr = {}
tpr = {}
roc_auc = {}

# Tạo một hình ảnh cho các đường ROC
plt.figure(figsize=(8, 6))

# Duyệt qua từng lớp
for i in range(len(class_names)):
    # Tính toán đường ROC và AUC cho lớp hiện tại
    fpr[i], tpr[i], _ = roc_curve(y_test_binary[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

    # Vẽ đường ROC cho lớp hiện tại
    plt.plot(fpr[i], tpr[i], lw=2, label=f'ROC curve for {class_names[i]} (area = {roc_auc[i]:.2f})')

# Cài đặt plot
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.savefig('efficientNetV2S/ROC_curve_10.png') # save before show. If not, it will save a blank image
plt.show()

# EfficientNetB4

In [ ]:
# Xây dựng mô hình
num_classes = 4
IMG_SIZE = 380
batch_size = 64
epochs = 50

inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = tf.keras.applications.efficientnet.EfficientNetB4(weights=None, include_top=True, classes=num_classes) 
predictions = base_model(inputs)

model = Model(inputs=inputs, outputs=predictions)

# Đóng băng các lớp của mô hình base
for layer in base_model.layers:
    layer.trainable = True

model.summary()

In [ ]:
# Đường dẫn đến thư mục chứa dữ liệu
data_dir = 'Data_Cucur_augmented/train'
test_dir = 'Data_Cucur_augmented/test'
# Thiết lập các thông số
validation_ratio = 0.1

# Sử dụng ImageDataGenerator để tạo dữ liệu và áp dụng các biến đổi
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=validation_ratio,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest'
)

# Tạo generator cho tập train và tập validation
train_data_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Chỉ sử dụng tập huấn luyện
)

val_data_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Chỉ sử dụng tập validation
)
test_datagen = ImageDataGenerator(rescale=1 / 255.0)
test_data_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False,
    seed=42
)

In [ ]:
print(len(train_data_generator.filepaths))
print(len(val_data_generator.filepaths))
print(len(test_data_generator.filepaths))

In [ ]:
#save log
class TimeTrainCSVLogger(tf.keras.callbacks.CSVLogger):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
        super().on_epoch_begin(epoch, logs)
    
    def on_epoch_end(self, epoch, logs=None):
        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - self.epoch_start_time
        logs['time_per_epoch'] = epoch_duration
        super().on_epoch_end(epoch, logs)
        print(f"Epoch {epoch + 1} - Training Time: {epoch_duration:.2f} seconds")
        
class TrainingTimeCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        super().on_train_begin(logs)

    def on_train_end(self, logs={}):
        end_time = time.time()
        duration = end_time - self.start_time
        super().on_train_end(logs)
        hours = duration // 3600
        minutes = (duration - (hours * 3600)) // 60
        seconds = duration - ((hours * 3600) + (minutes * 60))
        
        msg = f'Total Training Time: {int(hours)} hours, {int(minutes)} minutes, {seconds:.2f} seconds'
        print(msg)

        # Ghi tổng thời gian huấn luyện vào tệp CSV
        with open('efficientNetB4/log/train_logger11.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Total Training Time (seconds)'])
            writer.writerow([duration])
            writer.writerow(['Total Training Time (hours, minutes, seconds)'])
            writer.writerow([hours, minutes, seconds])

csv_logger = TimeTrainCSVLogger('efficientNetB4/log/train_logger11.csv', append=True, separator='\t')

checkpoint_filepath = "efficientNetB4/checkpoint/cp11.ckpt"
directory_checkpoint = os.path.dirname(checkpoint_filepath)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose=2,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=False,
    save_freq='epoch')


In [ ]:
# Optimizer và loss function
loss = CategoricalCrossentropy()
optimizer = Adam(learning_rate=0.0001)
model.compile(loss=loss, optimizer=optimizer, metrics= ['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

# Tính toán số bước cho tập train và tập validation
steps_per_epoch = train_data_generator.samples // batch_size
validation_steps = val_data_generator.samples // batch_size

history = model.fit(train_data_generator,
          steps_per_epoch=steps_per_epoch,
          validation_steps= validation_steps, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=val_data_generator, 
          callbacks=[model_checkpoint_callback,csv_logger,TrainingTimeCallback()]
          )

In [ ]:
# Đánh giá mô hình trên tệp test
test_loss, test_accuracy, test_recall, test_precision = model.evaluate(test_data_generator)

print(f'Test_loss: {test_loss}')
print(f'Test_accuracy: {test_accuracy}')
print(f'Test_recall: {test_recall}')
print(f'Test_precision: {test_precision}')

with open('efficientNetB4/Test_data_metrics11.txt', 'w') as file:
    file.write(f'Test_loss: {test_loss}\n')
    file.write(f'Test_accuracy: {test_accuracy}\n')
    file.write(f'Test_recall: {test_recall}\n')
    file.write(f'Test_precision: {test_precision}\n')

# Lưu mô hình
model.save('efficientNetB4/efficientNetB4_model.h5')

In [ ]:
# Lấy thông tin về độ chính xác trên tập huấn luyện và tập validation
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Vẽ đồ thị biểu diễn sự thay đổi của accuracy
plt.plot(range(1, epochs + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, epochs + 1), val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')
plt.legend()
plt.savefig('efficientNetB4/accuracy_vs_epochs_11.png') # save before show. If not, it will save a blank image
plt.show()

# Lấy thông tin về loss trên tập huấn luyện và tập validation
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Vẽ đồ thị biểu diễn sự thay đổi của loss
plt.plot(range(1, epochs + 1), train_loss, label='Training Loss')
plt.plot(range(1, epochs + 1), val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')
plt.legend()
plt.savefig('efficientNetB4/loss_vs_epochs_11.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'precision' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Precision vs Epochs')
plt.legend()
plt.savefig('efficientNetB4/precision_vs_epochs_11.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('efficientNetB4/recall_vs_epochs_11.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('efficientNetB4/recall_vs_epochs_11.png') # save before show. If not, it will save a blank image
plt.show()

# Dự đoán trên tập kiểm tra và lấy nhãn dự đoán
y_pred = model.predict(test_data_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Lấy nhãn thật của tập kiểm tra
y_true_indices = test_data_generator.classes

# Lấy danh sách các lớp từ tên thư mục trong val_data_generator
class_names = list(test_data_generator.class_indices.keys())

# Sử dụng chỉ số thứ tự để lấy tên của các lớp từ danh sách tên lớp
y_true_classes = [class_names[i] for i in y_true_indices]

# Tính toán confusion matrix
cm = confusion_matrix(y_true_indices, y_pred_labels)

# Hiển thị confusion matrix bằng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('efficientNetB4/confusion_matrix_11.png') # save before show. If not, it will save a blank image
plt.show()

# Tính classification report
class_names = list(test_data_generator.class_indices.keys())
report = classification_report(y_true_indices, y_pred_labels, target_names=class_names, output_dict=True)

# Tạo bảng classification report
report_table = []
for class_name, metrics in report.items():
    if class_name in class_names:
        row = [class_name, metrics['precision'], metrics['recall'], metrics['f1-score']]
        report_table.append(row)

headers = ["Class", "Precision", "Recall", "F1-Score"]
report_text = tabulate(report_table, headers, tablefmt="pretty")
# Lưu bảng classification report vào tệp văn bản
with open('efficientNetB4/classification_report_11.txt', 'w') as file:
    file.write(report_text)

# In bảng classification report ra màn hình
print(report_text)

# Chuyển đổi nhãn thành định dạng mã hóa one-hot
y_test_binary = label_binarize(test_data_generator.labels, classes=range(len(class_names)))

# Khởi tạo một từ điển trống để lưu FPR và TPR cho mỗi lớp
fpr = {}
tpr = {}
roc_auc = {}

# Tạo một hình ảnh cho các đường ROC
plt.figure(figsize=(8, 6))

# Duyệt qua từng lớp
for i in range(len(class_names)):
    # Tính toán đường ROC và AUC cho lớp hiện tại
    fpr[i], tpr[i], _ = roc_curve(y_test_binary[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

    # Vẽ đường ROC cho lớp hiện tại
    plt.plot(fpr[i], tpr[i], lw=2, label=f'ROC curve for {class_names[i]} (area = {roc_auc[i]:.2f})')

# Cài đặt plot
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.savefig('efficientNetB4/ROC_curve_11.png') # save before show. If not, it will save a blank image
plt.show()

# EfficientNetB5

In [ ]:
# Xây dựng mô hình
num_classes = 4
IMG_SIZE = 456
batch_size = 64
epochs = 50

inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = tf.keras.applications.efficientnet.EfficientNetB5(weights=None, include_top=True, classes=num_classes) 
predictions = base_model(inputs)

model = Model(inputs=inputs, outputs=predictions)

# Đóng băng các lớp của mô hình base
for layer in base_model.layers:
    layer.trainable = True

model.summary()

In [ ]:
# Đường dẫn đến thư mục chứa dữ liệu
data_dir = 'Data_Cucur_augmented/train'
test_dir = 'Data_Cucur_augmented/test'
# Thiết lập các thông số
validation_ratio = 0.1

# Sử dụng ImageDataGenerator để tạo dữ liệu và áp dụng các biến đổi
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=validation_ratio,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest'
)

# Tạo generator cho tập train và tập validation
train_data_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Chỉ sử dụng tập huấn luyện
)

val_data_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Chỉ sử dụng tập validation
)
test_datagen = ImageDataGenerator(rescale=1 / 255.0)
test_data_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False,
    seed=42
)

In [ ]:
print(len(train_data_generator.filepaths))
print(len(val_data_generator.filepaths))
print(len(test_data_generator.filepaths))

In [ ]:
#save log
class TimeTrainCSVLogger(tf.keras.callbacks.CSVLogger):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
        super().on_epoch_begin(epoch, logs)
    
    def on_epoch_end(self, epoch, logs=None):
        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - self.epoch_start_time
        logs['time_per_epoch'] = epoch_duration
        super().on_epoch_end(epoch, logs)
        print(f"Epoch {epoch + 1} - Training Time: {epoch_duration:.2f} seconds")
        
class TrainingTimeCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        super().on_train_begin(logs)

    def on_train_end(self, logs={}):
        end_time = time.time()
        duration = end_time - self.start_time
        super().on_train_end(logs)
        hours = duration // 3600
        minutes = (duration - (hours * 3600)) // 60
        seconds = duration - ((hours * 3600) + (minutes * 60))
        
        msg = f'Total Training Time: {int(hours)} hours, {int(minutes)} minutes, {seconds:.2f} seconds'
        print(msg)

        # Ghi tổng thời gian huấn luyện vào tệp CSV
        with open('efficientNetB5/log/train_logger12.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Total Training Time (seconds)'])
            writer.writerow([duration])
            writer.writerow(['Total Training Time (hours, minutes, seconds)'])
            writer.writerow([hours, minutes, seconds])

csv_logger = TimeTrainCSVLogger('efficientNetB5/log/train_logger12.csv', append=True, separator='\t')

checkpoint_filepath = "efficientNetB5/checkpoint/cp12.ckpt"
directory_checkpoint = os.path.dirname(checkpoint_filepath)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose=2,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=False,
    save_freq='epoch')


In [ ]:
# Optimizer và loss function
loss = CategoricalCrossentropy()
optimizer = Adam(learning_rate=0.0001)
model.compile(loss=loss, optimizer=optimizer, metrics= ['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

# Tính toán số bước cho tập train và tập validation
steps_per_epoch = train_data_generator.samples // batch_size
validation_steps = val_data_generator.samples // batch_size

history = model.fit(train_data_generator,
          steps_per_epoch=steps_per_epoch,
          validation_steps= validation_steps, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=val_data_generator, 
          callbacks=[model_checkpoint_callback,csv_logger,TrainingTimeCallback()]
          )

In [ ]:
# Đánh giá mô hình trên tệp test
test_loss, test_accuracy, test_recall, test_precision = model.evaluate(test_data_generator)

print(f'Test_loss: {test_loss}')
print(f'Test_accuracy: {test_accuracy}')
print(f'Test_recall: {test_recall}')
print(f'Test_precision: {test_precision}')

with open('efficientNetB5/Test_data_metrics.txt', 'w') as file:
    file.write(f'Test_loss: {test_loss}\n')
    file.write(f'Test_accuracy: {test_accuracy}\n')
    file.write(f'Test_recall: {test_recall}\n')
    file.write(f'Test_precision: {test_precision}\n')

# Lưu mô hình
model.save('efficientNetB5/efficientNetB5_model.h5')

In [ ]:
# Lấy thông tin về độ chính xác trên tập huấn luyện và tập validation
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Vẽ đồ thị biểu diễn sự thay đổi của accuracy
plt.plot(range(1, epochs + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, epochs + 1), val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')
plt.legend()
plt.savefig('efficientNetB5/accuracy_vs_epochs_12.png') # save before show. If not, it will save a blank image
plt.show()

# Lấy thông tin về loss trên tập huấn luyện và tập validation
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Vẽ đồ thị biểu diễn sự thay đổi của loss
plt.plot(range(1, epochs + 1), train_loss, label='Training Loss')
plt.plot(range(1, epochs + 1), val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')
plt.legend()
plt.savefig('efficientNetB5/loss_vs_epochs_12.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'precision' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Precision vs Epochs')
plt.legend()
plt.savefig('efficientNetB5/precision_vs_epochs_12.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('efficientNetB5/recall_vs_epochs_12.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('efficientNetB5/recall_vs_epochs_12.png') # save before show. If not, it will save a blank image
plt.show()

# Dự đoán trên tập kiểm tra và lấy nhãn dự đoán
y_pred = model.predict(test_data_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Lấy nhãn thật của tập kiểm tra
y_true_indices = test_data_generator.classes

# Lấy danh sách các lớp từ tên thư mục trong val_data_generator
class_names = list(test_data_generator.class_indices.keys())

# Sử dụng chỉ số thứ tự để lấy tên của các lớp từ danh sách tên lớp
y_true_classes = [class_names[i] for i in y_true_indices]

# Tính toán confusion matrix
cm = confusion_matrix(y_true_indices, y_pred_labels)

# Hiển thị confusion matrix bằng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('efficientNetB5/confusion_matrix_12.png') # save before show. If not, it will save a blank image
plt.show()

# Tính classification report
class_names = list(test_data_generator.class_indices.keys())
report = classification_report(y_true_indices, y_pred_labels, target_names=class_names, output_dict=True)

# Tạo bảng classification report
report_table = []
for class_name, metrics in report.items():
    if class_name in class_names:
        row = [class_name, metrics['precision'], metrics['recall'], metrics['f1-score']]
        report_table.append(row)

headers = ["Class", "Precision", "Recall", "F1-Score"]
report_text = tabulate(report_table, headers, tablefmt="pretty")
# Lưu bảng classification report vào tệp văn bản
with open('efficientNetB5/classification_report_12.txt', 'w') as file:
    file.write(report_text)

# In bảng classification report ra màn hình
print(report_text)

# Chuyển đổi nhãn thành định dạng mã hóa one-hot
y_test_binary = label_binarize(test_data_generator.labels, classes=range(len(class_names)))

# Khởi tạo một từ điển trống để lưu FPR và TPR cho mỗi lớp
fpr = {}
tpr = {}
roc_auc = {}

# Tạo một hình ảnh cho các đường ROC
plt.figure(figsize=(8, 6))

# Duyệt qua từng lớp
for i in range(len(class_names)):
    # Tính toán đường ROC và AUC cho lớp hiện tại
    fpr[i], tpr[i], _ = roc_curve(y_test_binary[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

    # Vẽ đường ROC cho lớp hiện tại
    plt.plot(fpr[i], tpr[i], lw=2, label=f'ROC curve for {class_names[i]} (area = {roc_auc[i]:.2f})')

# Cài đặt plot
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.savefig('efficientNetB5/ROC_curve_12.png') # save before show. If not, it will save a blank image
plt.show()